## Part 1

In [18]:
import os
import dateparser
import re
from bs4 import BeautifulSoup
import pandas as pd

In [19]:
def get_property_int(property_listing: str, element_type:str, class_name: str):
    property_element = property_listing.find(element_type, {"class": class_name})
    property_text = property_element.text if property_element else ""
    property_number = re.findall(r'\d+', property_text.replace('\xa0', ''))
    return int(''.join(property_number)) if property_number else 0

In [20]:

data = []
folder = "./kungalv_slutpriser/"

for filename in os.listdir(folder):
    if filename.endswith(".html"):
        print(filename)
        with open(f"{folder}{filename}", "r") as f:
            soup = BeautifulSoup(f, "html.parser")
            
            property_listings = soup.find_all("div", {"class": "qa-sale-card hcl-grid hcl-grid--columns-1 hcl-grid--md-columns-2"})
            
            for property_listing in property_listings:
                date_of_sale = property_listing.find("span", {"class": "hcl-label hcl-label--state hcl-label--sold-at"}).text.replace("Såld ", "")
                address = property_listing.find("h2", {"class": "sold-property-listing__heading qa-selling-price-title hcl-card__title"}).text
                location = property_listing.find("div", {"class": "sold-property-listing__location"}).div.text
                area_div = property_listing.find("div", {"class": "sold-property-listing__subheading sold-property-listing__area"})
                plot_area = get_property_int(property_listing, "div", "sold-property-listing__land-area")
                closing_price = get_property_int(property_listing, "span", "hcl-text hcl-text--medium")

                # Handle both formats for area. And if there is no area then set to 0
                if area_div.span:
                    boarea, biarea = area_div.text.replace(',', '.').split("+")
                    totalarea = float(boarea) + float(biarea.split()[0])
                elif len(area_div.text.split()) > 0:
                    totalarea = float(area_div.text.replace(',', '.').split()[0])
                else:
                    totalarea = 0

                # Store information in a dictionary and append to list
                data.append({
                    "Date of Sale": dateparser.parse(date_of_sale.strip()),
                    "Address": address.strip(),
                    "Location": ' '.join(location.split()),
                    "Plot Area": plot_area,
                    "Closing Price": closing_price,
                    "Total Area": totalarea
                })

# Convert list to DataFrame
df = pd.DataFrame(data)

# Export DataFrame to CSV file
df.to_csv("kungalv_slutpriser.csv", index=False)

kungalv_slutpris_page_03.html
kungalv_slutpris_page_20.html
kungalv_slutpris_page_27.html
kungalv_slutpris_page_24.html
kungalv_slutpris_page_26.html
kungalv_slutpris_page_06.html
kungalv_slutpris_page_32.html
kungalv_slutpris_page_01.html
kungalv_slutpris_page_10.html
kungalv_slutpris_page_02.html
kungalv_slutpris_page_17.html
kungalv_slutpris_page_25.html
kungalv_slutpris_page_07.html
kungalv_slutpris_page_04.html
kungalv_slutpris_page_40.html
kungalv_slutpris_page_36.html
kungalv_slutpris_page_38.html
kungalv_slutpris_page_16.html
kungalv_slutpris_page_18.html
kungalv_slutpris_page_11.html
kungalv_slutpris_page_31.html
kungalv_slutpris_page_22.html
kungalv_slutpris_page_14.html
kungalv_slutpris_page_33.html
kungalv_slutpris_page_05.html
kungalv_slutpris_page_30.html
kungalv_slutpris_page_09.html
kungalv_slutpris_page_29.html
kungalv_slutpris_page_12.html
kungalv_slutpris_page_34.html
kungalv_slutpris_page_35.html
kungalv_slutpris_page_23.html
kungalv_slutpris_page_13.html
kungalv_sl

## Part 2